In [4]:
import warnings 
warnings.filterwarnings("ignore")
import numpy as np
import math
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
import networkx as nx
from matplotlib import pyplot as plt
import pandas as pd
import sys
import os 
from datetime import datetime
import time
import itertools
import json

date = datetime.strptime('2020-07-25', "%Y-%m-%d") 
age = 55
number = 40
fieldID = '1234123'
symptom = []
# date,age,number,'1535123233',['发烧','咳嗽']
# date = datetime.strptime(sys.argv[1], "%Y-%m-%d")
# age = int(sys.argv[2])
# number = sys.argv[3]
# if str.isdigit(number): number = int(sys.argv[3])
# else: number = 50
# fieldID = sys.argv[4]
# symptom = sys.argv[5].split(',')


def dict_to_json(dic,filename):
    data = json.dumps(dic)
    file= open(filename, 'w')
    file.write(data)
    file.close()
    
def read_json(filename):
    with open(filename,'r') as file:
        load_dict = json.load(file)
        return load_dict    
    

def create_cpd_upplim(disease_table,base):
    res = {}
    for i in disease_table:
        if i not in res:
            res[i] = []
            print(i)
        #             print(i,disease_table[i])
            for j in itertools.product([1, 0], repeat = len(disease_table[i])):
            #             print(disease[i])
            #             print(j)
            #             print(np.array(list(disease_table[i].values())) , np.array(j))
        #               print(np.array(list(disease_table[i].values())),i,list(disease_table[i].keys()))
                temp = (np.array(list(disease_table[i].values())) * np.array(j)).max()
#                       print(temp,res[i],base[i])
                if temp == 0: res[i].append(base[i])
                else: res[i].append(temp)
    return res
def create_cpd_risk(disease_table,base):
    res = {}
    for i in disease_table:
        if i not in res:
            res[i] = []
        #             print(i,disease_table[i])
            for j in itertools.product([1, 0], repeat = len(disease_table[i])):
            #             print(disease[i])
            #             print(j)
            #             print(np.array(list(disease_table[i].values())) , np.array(j))
        #               print(np.array(list(disease_table[i].values())),i,list(disease_table[i].keys()))
                temp = (np.array(list(disease_table[i].values())) * np.array(j)).sum()
#                       print(temp,res[i],base[i])
                res[i].append(base[i]/math.exp(-temp))
    return res
def disease_table(disease):
    disease_list = {}
    for i in disease:
        for j in disease[i]:
            if j not in disease_list:
                disease_list[j] = {i:disease[i][j]}
            else: disease_list[j][i] = disease[i][j]
    return disease_list
# Myc ,0.25,0.1,0.1,0.1,0.1,0.1,0.1,0.1
# Salm ,0.25,0.1,0.1,0.1,0.1,0.1,0.1,0.1
#  HPS SS,0.25,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.25,0.1,0.1,0.1,0.1,0.1,0.1,0.1
class BayesN():
    def __init__(self,cause_d,disease_s,cpd_symptom,cpd_disease):
#         treatment,medrule data rearrangement
        self.sample = pd.read_excel('sample.xlsx')
        cause_disease = []
        disease_symptom = []
        for i in cause_d:
            for j in list(cause_d[i].keys()):
                cause_disease.append((j,i))
        self.cause = pd.DataFrame(cause_disease).rename({0:'cause',1:'disease'},axis = 1)
        for i in disease_s:
            for j in list(disease_s[i].keys()):
                disease_symptom.append((i,j))
        nodes = cause_disease
        
#         get all the nodes for baysian network
        nodes.extend(disease_symptom)
#         set up the model frame
        print('nonodd')
        self.model = BayesianModel(nodes)
#         list of disease_to_symptom
        self.symptom = pd.DataFrame(disease_symptom).rename({0:'disease',1:'symptom'},axis = 1)
        self.n_symptom = self.symptom.symptom.drop_duplicates().shape[0]
#         fillup all the cpds for symptom-nodes
#         print(self.symptom)
#         print(disease_s)
        for i in cpd_symptom:
            evi = list(self.symptom[self.symptom.symptom == i].disease)
#             print(i,evi)
#             print(evi)
            self.model.add_cpds(TabularCPD(variable = i , variable_card = 2,values = [cpd_symptom[i], [1-j for j in cpd_symptom[i]]],evidence = evi,evidence_card = [2]*len(evi)))
#         fillup all the cpds for disease-nodes
        for i in cpd_disease:
            evi = list(self.cause[self.cause.disease == i].cause)
#             print(i,evi)
            self.model.add_cpds(TabularCPD(variable = i , variable_card = 2,values = [cpd_disease[i], [1-j for j in cpd_disease[i]]],evidence = evi,evidence_card = [2]*len(evi)))
#         fillup all the cpds for cause_nodes(meaningless in calculation cuz those nodes are required information, no update needed)
        for i in self.cause.cause.drop_duplicates():
            print('staaaa')
            if i != 'PRRS':
                self.model.add_cpds(TabularCPD(variable=i,variable_card = 2,
                            values=[
                                [0],
                                [1]
                            ]))
#         print(self.model.check_model())
    def predict(self,date,age,number,fieldID,symp):
        self.symp = symp
        if number <10: self.method = ['both']
        else: self.method = ['group']
        if (field.fieldname == fieldID).sum() !=0: field_info = field[field.fieldname == fieldID].drop('fieldname',axis = 1)
        else: field_info = pd.DataFrame([[1]*(field.shape[1]-1)],columns = field.columns[1:])
        season_age_info = []
#        base_info including age and season for all disease
        cause = list(self.cause.cause.drop_duplicates())
        season_age_info = pd.DataFrame([[1]*len(cause)], columns = cause)
        season_age_info.Age_RV = 1-(age<=42)
        season_age_info.Age_Ecoli = 1-((age<=35)&(age>=28))
        season_age_info.Age_clostridiu = 1-(age <=35)
        season_age_info.Season_PED = 1-((date.month <5)|(date.month > 11))
        season_age_info.Season_RV = 1-((date.month <4)|(date.month > 10))
        season_age_info.Season_Salm = 1-((date.month <11)&(date.month > 7))
        season_age_info.Season_clostridiu = 1-((date.month <6)&(date.month > 2))
        non_symp = list(self.symptom.symptom.drop_duplicates()[self.symptom.symptom.drop_duplicates().apply(lambda x: x not in symp)])
        symp = pd.DataFrame([[0]*len(symp)],columns = symp)
        non_symp = pd.DataFrame([[1]*len(non_symp)],columns = non_symp)
        obs = pd.concat([season_age_info,symp,non_symp],axis = 1)
        if (obs[['腹泻-褐色/黑色/沥青色稀粪','腹泻-棕/血色稀粪','腹泻-绿色稀粪','腹泻-水样稀粪','粪便带血/血痢','腹泻-白/灰色稀粪','腹泻-黄色稀粪']]==0).sum(axis = 1).reset_index().rename({0:'res'},axis = 1).iloc[0,1]!= 0:
            non_symp = list(self.symptom.symptom.drop_duplicates()[self.symptom.symptom.drop_duplicates().apply(lambda x: x not in symp and (x not in ['黄色水样','灰色水样或糊状','绿色水样','白色糊状','暗棕色水样','红/红棕色/便血']))])
#             print(non)
            non_symp = pd.DataFrame([[1]*len(non_symp)],columns = non_symp)
            obs = pd.concat([season_age_info,symp,non_symp],axis = 1)
#         print(obs.transpose())
        print('start')
#         return self.model.predict(obs)

#         result = self.model.predict_probability(obs)[['PED_0','RV_0','Ecoli_0','Salm_0','clostridiu_C_0','PRRS_0','Mhyo_0','PCV_0','HPS_0','SS_0','Pm_0']]
        result = self.model.predict_probability(obs)
        result.columns = list(self.cause.disease.drop_duplicates())
        result = result.transpose().rename({0:'disease'},axis =1)
        result = result[result.disease*100 >= 0].sort_values('disease',ascending = False).head(5)
#         result.disease = result.disease.apply(lambda x: str(round(x*100,1))+'%') 
        print(result.disease.apply(lambda x: str(round(x*100,1))+'%'))
        self.disease_candidate = list(result.disease.sort_values(ascending = False).head(3).index)
        return result.disease.apply(lambda x: str(round(x*100,1))+'%')
#     def treat(self, disease_candidate = [],method = []):
#         if disease_candidate == []:
#             self.recipe = self.medicine[self.medicine.disease.apply(lambda x: x in self.disease_candidate)]
#         else:
#             self.recipe = self.medicine[self.medicine.disease.apply(lambda x: x in disease_candidate)]
#         if method ==[]:method = self.method
#         if method == 'group' :self.recipe = self.recipe[self.recipe.method.apply(lambda x: x not in ['粉针','注射液'])]
#         self.recipe.to_excel('res.xlsx')
#         print(self.recipe)
    
    def sample_recom(self):
        res = []
#         print(self.symp)
        for i in self.symp:
            res.append(self.sample[self.sample.symptom.apply(lambda x: i in x.split(','))][['sample','sus']])
        if res == []: return 
        res = pd.concat(res).reset_index().drop('index',axis =1)
        res.columns = ['样品采集','检测项目']
        print(res)
        return res
#         self.recipe.med_ingredient 
base_symptom = read_json('base_symptom0826.json')
base_disease = read_json('base_disease0826.json')
disease_symptom = read_json('disease_symptom0826.json')
cause_disease = read_json('cause_disease0826.json')


field = []
for i in list(cause_disease.values()):
    field.extend(list(i.keys()))
field = pd.DataFrame([[1]*len(field)]*4,columns = field)
field.index = ['15358233','15358234','15358235','15358236']
field = field.reset_index().rename({'index':'fieldname'},axis = 1)
cpd_symptom = create_cpd_upplim(disease_table(disease_symptom),base_symptom)
cpd_disease = create_cpd_risk(cause_disease,base_disease)

starttime = time.time()
print('sssss')
a = BayesN(cause_disease,disease_symptom,cpd_symptom,cpd_disease)
# k = a.predict(date,age,number,'1535123233',['发烧','咳嗽'])
res = {}
symptom = ['震颤/惊厥','跛行/僵硬/关节炎','神经/角弓反张/划水','咳嗽','鼻甲变形']
res['disease'] =  a.predict(date,age,number,fieldID,symptom)
# a.treat()
res['sample_recom']=a.sample_recom()
print(time.time() -starttime)
# disease_candidate = k.disease.sort_values(ascending = False).head(3).index
# sys.exit(res)


腹泻-黄色稀粪
腹泻-白/灰色稀粪
腹泻-绿色稀粪
腹泻-水样稀粪
呕吐
粪便带血/血痢
脱水/眼窝凹陷
毛粗乱/毛耸/脱毛/结痂/皱褶/龟裂
发烧
咳嗽
喘气
呼吸困难
肺炎
震颤/惊厥
运动失调
跛行/僵硬/关节炎
神经/角弓反张/划水
发抖/震颤/抖抖
瘫痪
贫血苍白
结膜炎
体表发紫/发红/发黄/发白
水肿
出血、淤血（指压不褪色）/充血（指压褪色）
咬耳/烂耳朵
突然死亡
无力/犬坐
粪便粘液/痢疾
流产
无乳
喷嚏/鼻涕
口吐白沫
乱撞/尖叫
口鼻出血/血沫
腹泻-棕/血色稀粪
便秘
皮肤斑块/糜烂/溃疡/擦伤
弱仔/死胎
脱肛/外阴红肿
脓包/鳞屑/结痂/糠疹/丘疹
增生性皮炎/皮肤发育不全
不发情
木乃伊胎
空怀/不孕
总仔数少
先天震颤
返情
阴部红肿/发紫
阴道恶露/僵尸胎
斑点胎
鼻、蹄、趾溃烂/水泡
乳汁不下
腹泻-褐色/黑色/沥青色稀粪
鼻甲变形
泪斑
腹胀
sssss
nonodd
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
staaaa
s

KeyboardInterrupt: 

In [15]:
def create_cpd_upplim(disease_table,base):
    res = {}
    for i in disease_table:
        print(i)
        if i not in res:
            res[i] = []
        #             print(i,disease_table[i])
            for j in itertools.product([1, 0], repeat = len(disease_table[i])):
            #             print(disease[i])
            #             print(j)
            #             print(np.array(list(disease_table[i].values())) , np.array(j))
        #               print(np.array(list(disease_table[i].values())),i,list(disease_table[i].keys()))
                temp = (np.array(list(disease_table[i].values())) * np.array(j)).max()
#                       print(temp,res[i],base[i])
                if temp == 0: res[i].append(base[i])
                else: res[i].append(temp)
    return res

In [3]:
res['disease']

KeyError: 'disease'